In [2]:
# %load symmetric_encryption.py
#!/usr/bin/env/python3

import urllib.request
from Crypto.Cipher import AES
import binascii
import base64
import random
import os
import zlib

################################################################################
# CS 284 Padding Utility Functions
################################################################################

# s is a bytearray to pad, k is blocklength
# you won't need to change the block length
def cmsc284pad(s,k=16):
  if k > 255:
    print("pkcs7pad: padding block length must be less than 256")
    return bytearray()
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s.extend([i])
  return s

# s is bytes to pad, k is blocklength
# you won't need to change the block length
def cmsc284padbytes(s,k=16):
  if k > 255:
    raise Exception("pkcs7pad: padding block length must be less than 256")
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s += chr(i).encode("utf-8")
  return s

# s is bytes to unpad, k is blocklength
# you won't need to change the block length
def cmsc284unpad(s,k=16):
  if not cmsc284checkpadding(s,k):
    print("cmsc284unpad: invalid padding")
    return b''
  n = s[len(s)-1]
  return s[:len(s)-n]

# checks padding on s and returns a boolean
# you won't need to change the block length
def cmsc284checkpadding(s,k=16):
  if(len(s) == 0):
    #print("Invalid padding: String zero length"%k) 
    return False
  if(len(s)%k != 0): 
    #print("Invalid padding: String is not multiple of %d bytes"%k) 
    return False
  n = s[len(s)-1]
  if n > k or n == 0:
    return False
  else: 
    for i in range(n):
      if s[len(s)-1-i] != (n-i):
        return False
  return True

################################################################################
# Function for querying the server
################################################################################

PPS2SERVER = "http://cryptoclass.cs.uchicago.edu/"
def make_query(task, cnetid, query):
  DEBUG = False
  if DEBUG:
    print("making a query")
    print("Task:", task)
    print("CNET ID:", cnetid)
    print("Query:", query)
  if (type(query) is bytearray) or (type(query) is bytes):
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
  else:
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
  if DEBUG:
    print("Querying:", url)

  with urllib.request.urlopen(url) as response:
    raw_answer = response.read()
    answer = base64.urlsafe_b64decode(raw_answer)
    if DEBUG:
      print("Answer:", answer)
    return answer
  return None


################################################################################
# Problem 1 SOLUTION
################################################################################

def problem1(cnetid):
  return b''


################################################################################
# Problem 2 SOLUTION
################################################################################

def problem2(cnetid):
  return b''


################################################################################
# Problem 3 SOLUTION
################################################################################

def problem3(cnetid):
  return b''


################################################################################
# Problem 4 SOLUTION
################################################################################

def problem4(cnetid):
  return b''


################################################################################
# Problem 5 SOLUTION
################################################################################

def problem5(cnetid):
  return b''

################################################################################
# Problem 6 SOLUTION
################################################################################

def problem6(cnetid):
  return b''

In [3]:
from collections import Counter
import math

In [4]:
make_query('foura', 'ruolinzheng', '')

b'\x88[\x02s\x84\xbd\x1c\xfa&TL+bA\xf2\x8d\xcd\xa5\\I\xca\xca\xfd(\xc2\xb2F\xaaP\xa3\x15\x12K\xa8\x9fsC\x81\xd125\x86\xd2\x7f\xb6\x87QH8\x15X\xbc\xeb\xcb\x16L\xbf\xc4\x97\x02{f\xa8\n\xfe\x8b\x08\xb0{\xc0Y\x05\x16Z\x19R\xf1\xd69p'

In [10]:
ctextb = make_query('fourb', 'ruolinzheng', bytes(32))

In [12]:
m1, m2 = ctextb[:16], ctextb[16:]

In [14]:
key = bytes([b1 ^ b2 for b1, b2 in zip(m1, m2)])

In [20]:
cipher = AES.new(key, AES.MODE_ECB)
query = cipher.encrypt(bytes(16))
make_query('fourc', 'ruolinzheng', query)

b'Permission denied'

In [25]:
def xor_bytes(bytes1, bytes2):
  if len(bytes1) != len(bytes2):
    raise
  return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])

In [45]:
def problem4_enc(key, msg):
  padded_msg = cmsc284padbytes(msg)
  parsed_msgs = [padded_msg[i : i + 16] for i in 
                 range(0, len(padded_msg), 16)]
  ctexts = [key]
  cipher = AES.new(key, AES.MODE_ECB)
  for i in range(len(parsed_msgs)):
    to_encrypt = xor_bytes(ctexts[i], parsed_msgs[i])
    ctexts.append(cipher.encrypt(to_encrypt))
  return b''.join(ctexts[1:]) # omit ctexts[0] which is the key

In [46]:
query = problem4_enc(key, b'let me in please')

In [47]:
make_query('fourb', 'ruolinzheng', query)

b'Message decrypted properly'

In [48]:
make_query('fourc', 'ruolinzheng', query)

b'Success!'

In [49]:
def get_key(cnetid):
  ctext_b = make_query('fourb', cnetid, bytes(32))
  m1, m2 = ctext_b[:16], ctext_b[16:]
  key = xor_bytes(m1, m2)
  return key

In [50]:
def problem4(cnetid):
  msg = b'let me in please'
  key = get_key(cnetid)
  query = problem4_enc(key, msg)
  return make_query('fourc', cnetid, query)

In [51]:
problem4('davidcash')

b'Success!'